In [1]:
import sys
print('python', sys.version)

import numpy as np
print('numpy', np.__version__)

import pandas as pd
print('pandas', pd.__version__)

import matplotlib as mpl
print('matplotlib', mpl.__version__)

import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as sci
import glob
import networkx as nx
import Bio.KEGG.KGML.KGML_parser as keg

import pickle
import matplotlib.patches as mpatches

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import cmapPy.pandasGEXpress.parse as cp
import cmapPy.pandasGEXpress.write_gctx as cw

python 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:27:36) [GCC 11.2.0]
numpy 1.26.4
pandas 2.2.3
matplotlib 3.10.0


# entrez to sym gene mapping

In [2]:
# mapping human gene symbol to entrez
dat=pd.read_table('data/Sym2Entrez.txt',sep='\t')
dat.columns=['sym','entrez']
dat=dat.loc[dat['entrez'].notnull()]
dat=dat.loc[dat['sym'].notnull()]
dat['entrez']=dat['entrez'].astype(int).astype(str)
s2e_dic=dat.set_index('sym')['entrez']
e2s_dic=dat.set_index('entrez')['sym']

# GSEA

## robust rank algorithm (RRA)

In [3]:
NCES_df=pd.read_csv("./results/Best_NCES_all_cell.csv", sep=',', index_col=0)

import numpy as np
import pandas as pd
from scipy.stats import rankdata
from scipy.special import betainc  # regularized incomplete beta I_x(a,b)

def robust_rank_aggregation(df: pd.DataFrame, higher_is_better: bool = True):
    """
    df: rows=genes, cols=cell lines, values=essentiality scores (e.g., NCES)
    higher_is_better: True if larger score = more essential

    Returns:
      result_df with columns:
        - rra_score: RRA minimum cumulative probability
        - pval 
        - fdr: Benjamini-Hochberg adjusted q-value
        - rra_rank: rra_score rank (1=top)
        - gsea_score: −log10 FDR
    """
    genes = df.index.to_numpy()
    m = df.shape[1]  # number of lists
    n = df.shape[0]  # number of items per list

    rank_mat = np.zeros_like(df.to_numpy(dtype=float))
    for j, col in enumerate(df.columns):
        col_values = df[col].to_numpy(dtype=float)

        nan_mask = np.isnan(col_values)

        arr_for_rank = -col_values if higher_is_better else col_values
        fill_val = np.nanmax(arr_for_rank) + 1 if not np.all(nan_mask) else 0.0
        arr_for_rank[nan_mask] = fill_val

        ranks = rankdata(arr_for_rank, method='average')
        ranks[nan_mask] = np.nan
        rank_mat[:, j] = ranks

    # 2) RRA computation for each gene
    rra_scores = np.empty(n, dtype=float)
    rra_scores.fill(np.nan)

    for i in range(n):
        ranks_i = rank_mat[i, :]
        ranks_i = ranks_i[~np.isnan(ranks_i)]
        if ranks_i.size == 0:
            rra_scores[i] = np.nan
            continue

        # observed N.
        m_i = ranks_i.size

        # normalized p = r/n 
        pvals = np.sort(ranks_i / n)

        # k=1..m_i ->F_{k:m_i}(p_(k)) = I_{p_(k)}(k, m_i-k+1)
        # betainc(a,b,x) = I_x(a,b)
        k_arr = np.arange(1, m_i + 1, dtype=int)
        a = k_arr.astype(float)
        b = (m_i - k_arr + 1).astype(float)

        F_k = betainc(a, b, pvals)

        rra_scores[i] = np.nanmin(F_k)

    # 3) BH-FDR
    def bh_fdr(p):
        p = np.asarray(p, dtype=float)
        mask = ~np.isnan(p)
        p_valid = p[mask]
        m_tests = p_valid.size
        if m_tests == 0:
            return np.full_like(p, np.nan)
        order = np.argsort(p_valid)
        ranked = p_valid[order]
        fdr = ranked * m_tests / (np.arange(1, m_tests + 1))

        fdr = np.minimum.accumulate(fdr[::-1])[::-1]

        fdr_full = np.full_like(p, np.nan)
        fdr_full[mask] = fdr[np.argsort(order)]

        return np.minimum(fdr_full, 1.0)

    pval = rra_scores.copy() 
    fdr = bh_fdr(pval)

    # 4) results summary
    eps = 1e-300
    gsea_score = -np.log10(np.maximum(fdr, eps))

    result_df = pd.DataFrame({
        "RRA_score": rra_scores,
        "pval": pval,
        "FDR": fdr,
        "GSEA_input": gsea_score
    }, index=genes).sort_values(["RRA_score", "FDR"], ascending=[True, True])

    result_df["RRA_rank"] = np.arange(1, result_df.shape[0] + 1)

    return result_df

result = robust_rank_aggregation(NCES_df, higher_is_better=True)
result

result.to_csv('results/RRA_results.csv', sep=',')

,RRA_score,pval,FDR,GSEA_input,RRA_rank
PLK1,1.059049e-22,1.059049e-22,1.897392e-18,1.772184e+01,1
CCNB1,2.316140e-19,2.316140e-19,2.074799e-15,1.468302e+01,2
TOP2A,4.980439e-17,4.980439e-17,2.974318e-13,1.252661e+01,3
XPO1,1.178631e-16,1.178631e-16,4.886364e-13,1.231101e+01,4
AURKA,1.363687e-16,1.363687e-16,4.886364e-13,1.231101e+01,5
...,...,...,...,...,...
USP29,9.996094e-01,9.996094e-01,9.998326e-01,7.271569e-05,17912
LRRIQ3,9.998281e-01,9.998281e-01,9.999956e-01,1.927751e-06,17913
PATE1,9.999890e-01,9.999890e-01,9.999999e-01,5.410847e-08,17914
C5orf58,9.999975e-01,9.999975e-01,9.999999e-01,5.410847e-08,17915


## GSEA algorithm

In [5]:
import pandas as pd
import numpy as np
import gseapy as gp
from pathlib import Path


# ----------------------------------
# 0) Input: RRA result DataFrame `rra_res`
#    Required column: 'GSEA_input' (higher value = higher rank)
#    Index: gene symbols or Entrez IDs (recommended: HGNC symbols)
# ----------------------------------

# 1) Prepare ranking vector (handle duplicates / NaNs)
def perform_GSEA(rra_res, gene_set_name):
    r = rra_res["GSEA_input"].copy()
    
    # Clean gene names (optional)
    r.index = r.index.astype(str).str.strip()
    
    # Remove NaN / Inf values
    r = r.replace([np.inf, -np.inf], np.nan).dropna()
    
    # Remove duplicated genes: keep the one with the highest score
    r = r[~r.index.duplicated(keep=False)].sort_values(ascending=False)
    
    # GSEA prerank expects a Series or a two-column rank file
    # → gseapy accepts Series directly, but saving a file improves reproducibility
    rank_file = Path("ranked_genes.rnk")
    r.to_csv(rank_file, sep="\t", header=False)
    
    # 3) Run preranked GSEA
    #    Note: min_size / max_size filter gene set size,
    #          permutation_num trades off speed vs reproducibility
    prerank_res = gp.prerank(
        rnk=str(rank_file),                  # File path or pandas.Series
        gene_sets=gene_set_name,             # Gene set name, GMT file, or dict
        processes=4,                         # Parallel processes
        permutation_num=1000,                # Recommended ≥1000 (use 100–500 for quick tests)
        outdir="gsea_go_bp",                 # Output directory (tables/plots saved)
        seed=42,
        min_size=10,
        max_size=5000,                       # GO:BP can be large, set a high upper bound
        format="png"                         # Summary plot format
    )
    
    # Sort by NES (positive enrichment first)
    res = prerank_res.res2d.sort_values("NES", ascending=False)
    
    # Select significantly enriched positive pathways
    sig = res[(res["NES"] > 0) & (res["FDR q-val"] < 0.05)]
    return sig

In [8]:
import pandas as pd
import numpy as np
import gseapy as gp
from pathlib import Path


# ----------------------------------
# 0) 입력: RRA 결과 DataFrame `result`
#    필요 컬럼: 'gsea_score' (값이 클수록 상위)
#    인덱스: gene symbol 또는 Entrez ID (권장: HGNC symbol)
# ----------------------------------

# 1) 랭킹 벡터 준비 (중복/NaN 정리)
def perform_GSEA(rra_res,  gene_set_name):
    r = rra_res["GSEA_input"].copy()
    
    # 유전자명 정리 (옵션)
    r.index = r.index.astype(str).str.strip()
    
    # NaN/Inf 제거
    r = r.replace([np.inf, -np.inf], np.nan).dropna()
    
    # 중복 유전자 제거: 스코어가 가장 큰 것만 유지
    r = r[~r.index.duplicated(keep=False)].sort_values(ascending=False)
    
    # GSEA prerank는 Series(or 2-col file)를 기대함
    # → gseapy는 Series도 직접 받지만, 재현성을 위해 파일로 저장해두는 것도 좋음
    rank_file = Path("ranked_genes.rnk")
    r.to_csv(rank_file, sep="\t", header=False)
    
    # 3) preranked GSEA 실행
    #    참고: min_size/max_size는 pathway 크기 필터, permutation_num은 재현성/속도 트레이드오프
    prerank_res = gp.prerank(
        rnk=str(rank_file),                  # 파일 경로 또는 pandas.Series 가능
        gene_sets=gene_set_name,             # 또는 로컬 .gmt 파일 경로/딕셔너리도 가능
        processes=4,                         # 병렬 처리
        permutation_num=1000,                # 추천: 1000 이상 (시간 오래 걸리면 100~500로 시작)
        outdir="gsea_go_bp",                 # 결과 폴더 (표/그림 자동 저장)
        seed=42,
        min_size=10,
        max_size=5000,                       # GO:BP는 항목이 커서 상한 넉넉히
        format="png"                         # 요약 플롯 포맷
    )
    
    res = prerank_res.res2d.sort_values("NES", ascending=False)  # NES 내림차순(양의 방향이 top)
    sig = res[(res["NES"] > 0) & (res["FDR q-val"] < 0.05)]
    return sig

In [7]:
rra_res=pd.read_table('results/RRA_results.csv', sep=',', index_col=0)
GSEA_sig_res=perform_GSEA(rra_res, "GO_Biological_Process_2023") 
GSEA_sig_res.to_csv('results/GSEA_GO_Biological_Process_2023_sig_res.csv', sep=',')
top10 = GSEA_sig_res.sort_values("NES", ascending=False).head(20)
for term in top10['Term']:
    term

/tmp/ipykernel_259756/1149600579.py:34: DeprecationWarning: processes is deprecated; use threads
  prerank_res = gp.prerank(
2025-12-19 01:48:17,588 [WARNING] Duplicated values found in preranked stats: 31.04% of genes
The order of those genes will be arbitrary, which may produce unexpected results.


'Regulation Of Meiotic Cell Cycle (GO:0051445)'

'Anaphase-Promoting Complex-Dependent Catabolic Process (GO:0031145)'

'Regulation Of Ubiquitin Protein Ligase Activity (GO:1904666)'

'Nuclear Transport (GO:0051169)'

...

'Nucleocytoplasmic Transport (GO:0006913)'

'Mitotic Cell Cycle Phase Transition (GO:0044772)'

'Regulation Of Exit From Mitosis (GO:0007096)'

'Regulation Of miRNA Transcription (GO:1902893)'

'Positive Regulation Of Chromosome Separation (GO:1905820)'

# 15 among top 30 genes (not included in GS)

In [4]:
available_cells= ["A549",'A375','BICR6','HT29','PC3','U251MG','MCF7']
# ----------------> load all_ess_score_res

gs_all_list=[]

###### TTD
with open("./data_preproc/gs_therTarget_TTD.pickle", 'rb') as file:
    TTD_gs_dic=pickle.load(file)

for cell in available_cells:
    gs_all_list+=TTD_gs_dic[cell]

with open("./data_preproc/gs_therTarget_drugbank.pickle", 'rb') as file:
    drugbank_gs_dic=pickle.load(file)

for cell in available_cells:
    gs_all_list+=drugbank_gs_dic[cell]

gs_all_list=[e2s_dic[ent] for ent in list(set(gs_all_list)) if ent in e2s_dic.keys()]

rra=pd.read_table('results/RRA_results.csv', sep=',', index_col=0)

rra['is_gs']=0
rra.loc[list(set(gs_all_list)&set(rra.index)), 'is_gs']=1

rra.to_csv('results/RRA_results_with_gs.csv', sep=',')

In [47]:
print(list(rra.head(30).loc[rra['is_gs']==0].index))

['CCNB1', 'SRSF2', 'TPR', 'WEE1', 'BRCA1', 'CDC7', 'PSMA7', 'U2AF1', 'PDPK1', 'CCNA2', 'UBE2I', 'CDC27', 'KMT2D', 'MCM2', 'BUB1']
